In [52]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle
import xgboost as xgb
from sklearn.ensemble import VotingClassifier

In [53]:
hogvectors_train = pickle.load(open('hogvectors_train.pkl', 'rb'))
hogvectors_test = pickle.load(open('hogvectors_test.pkl', 'rb'))

In [54]:
X_train_data = [hogfeature_Xtrain[0:8100] for hogfeature_Xtrain in hogvectors_train]
X_test_data = [hogfeature_Xtest[0:8100] for hogfeature_Xtest in hogvectors_test]

In [55]:
# ดึงข้อมูลทุกแถว แต่เอาแค่คอลัมน์สุดท้าย มาเป็น class
Y_train_data = [hogfeature_Ytrain[-1] for hogfeature_Ytrain in hogvectors_train]
Y_test_data = [hogfeature_Ytest[-1] for hogfeature_Ytest in hogvectors_test]

In [56]:
label_encoder = LabelEncoder() # สร้าง object label_encoder จาก Class LabelEncoder เพราะต้องใช้ในการแปลงชื่อยี่ห้อรถยนต์เป็นตัวเลข
y_cls_train = label_encoder.fit(Y_train_data) # ใช้ .fit(Y_train_data) เพื่อใช้ในการเรียนรู้ว่า ชื่อยี่ห้อรถยนต์จะถูกแทนด้วยตัวเลขอะไรบ้าง ( ทำ mapping )
y_labelNum_train = label_encoder.transform(Y_train_data) # หลังจากใช้ .fit เพื่ออบรมแล้วจะเป็นตัวเลขตาม mapping ที่ถูกสร้างไว้
y_cls_test = label_encoder.fit(Y_test_data)
y_labelNum_test = label_encoder.transform(Y_test_data)
# # แปลงชุดข้อมูล สตริง ยี่ห้อรถจาก Test เป็น TableIndex

In [57]:
# สร้าง object จาก model DecisionTree
clf = DecisionTreeClassifier(random_state=42)

In [58]:
# สร้าง object จาก modelXGBoost
xgb_model = xgb.XGBClassifier(objective="multi:softmax",num_class=len(label_encoder.classes_), random_state=42)

In [59]:
# ทำการรวม 2โมเดล ที่สร้างไว้มารวมด้วยกัน 
ensemble_model = VotingClassifier(estimators=[('DecisionTree', clf), ('XGBoost', xgb_model)], voting='hard',weights=[1, 4])

In [60]:
# .fit() X_train_data เป็นการให้โมเดลมันเรียนรู้ข้อมูลที่เหมาะสมกับข้อมูล ส่วน Y_train_data เป็นคำตอบที่ควรจะได้จากการเรียนรู้
ensemble_model.fit(X_train_data, y_labelNum_train) 

VotingClassifier(estimators=[('DecisionTree',
                              DecisionTreeClassifier(random_state=42)),
                             ('XGBoost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=N

In [61]:
# ใช้ข้อมูลทดสอบ X_test_data เพื่อไว้ทำนายผลลัพธ์ที่ได้จากโมเดลนี้ของข้อมูลทดสอบ
y_pred = ensemble_model.predict(X_test_data) 

In [62]:
accuracy = accuracy_score(y_labelNum_test, y_pred)
confusionMatrix = confusion_matrix(y_labelNum_test, y_pred)
print("Accuracy: ", accuracy)
print("Confusion Matrix: ", confusionMatrix)

Accuracy:  0.40860215053763443
Confusion Matrix:  [[ 8  4  1  1  3]
 [ 3  9  1  3  7]
 [ 3  3  2  4  0]
 [ 7  4  1 14  2]
 [ 2  1  2  3  5]]


In [63]:
path_model = 'model_genhog.pkl'
pickle.dump(ensemble_model, open(path_model, 'wb'))